# Module 2: LangGraph Fundamentals - HR Agent Labs

**Updated for LangChain 1.0 Alpha**

**Building on Module 1**: In Module 1, you built three HR agents using LangChain's high-level functions. Now we'll peek under the hood and learn LangGraph - the foundation that powers those agents.

**Why learn LangGraph?** Understanding LangGraph gives you:
- Full control over agent workflows
- Ability to create custom patterns
- Better debugging and optimization
- Foundation for advanced agentic systems

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install required packages
!pip install --pre -U langchain langchain-openai langgraph

## Setup: Configure OpenAI API Key

In [ ]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("✅ API Key configured!")

---
# Lab 1: Your First LangGraph - Linear Workflow

**Objective:** Build a simple linear workflow using LangGraph's core concepts: State, Nodes, Edges, START, and END.

**Scenario:** Create an employee onboarding workflow that processes new hires step by step.

**Key Concepts:**
- **State**: Data that flows through the graph
- **Nodes**: Functions that do work
- **Edges**: Connections between nodes
- **START**: Entry point
- **END**: Exit point

## Imports

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Imports successful!")

## Part 1: Understanding State

**State** is the data structure that flows through your graph. It's like a shared clipboard that every node can read and update.

In [ ]:
class OnboardingState(TypedDict):
    """State for employee onboarding workflow"""
    employee_name: str
    employee_id: str
    department: str
    messages: Annotated[list, add_messages]
    validation_status: str
    tasks_completed: list

print("✅ OnboardingState defined!")
print("\nState fields:")
print("  - employee_name: Employee's full name")
print("  - employee_id: Unique employee ID")
print("  - department: Department name")
print("  - messages: Conversation history")
print("  - validation_status: Validation result")
print("  - tasks_completed: List of completed tasks")

## Part 2: Creating Nodes

**Nodes** are functions that do work. Each node receives state, processes it, and returns updates.

In [ ]:
def validate_employee(state: OnboardingState):
    """Node 1: Validate employee information"""
    print(f"📋 Step 1: Validating employee {state['employee_name']}...")
    
    # Simple validation
    if state['employee_name'] and state['employee_id']:
        return {
            "validation_status": "✅ Validated",
            "messages": [("assistant", f"Employee {state['employee_name']} (ID: {state['employee_id']}) validated successfully")]
        }
    else:
        return {
            "validation_status": "❌ Failed",
            "messages": [("assistant", "Validation failed: Missing information")]
        }

def assign_equipment(state: OnboardingState):
    """Node 2: Assign equipment based on department"""
    print(f"💻 Step 2: Assigning equipment for {state['department']} department...")
    
    equipment = {
        "Engineering": ["Laptop (MacBook Pro)", "Monitor", "Keyboard", "Mouse"],
        "HR": ["Laptop (ThinkPad)", "Headset"],
        "Sales": ["Laptop (MacBook Air)", "Phone", "CRM Access"],
        "Marketing": ["Laptop (MacBook Pro)", "Design Software License"]
    }
    
    dept_equipment = equipment.get(state['department'], ["Standard Laptop"])
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Equipment assigned"],
        "messages": [("assistant", f"Assigned equipment: {', '.join(dept_equipment)}")]
    }

def setup_accounts(state: OnboardingState):
    """Node 3: Set up employee accounts"""
    print(f"🔐 Step 3: Setting up accounts for {state['employee_name']}...")
    
    accounts = ["Email", "Slack", "HR Portal", "Leave Management System"]
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Accounts created"],
        "messages": [("assistant", f"Created accounts: {', '.join(accounts)}")]
    }

def send_welcome_email(state: OnboardingState):
    """Node 4: Send welcome email"""
    print(f"📧 Step 4: Sending welcome email...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Welcome email sent"],
        "messages": [("assistant", f"Welcome email sent to {state['employee_name']}@company.com")]
    }

print("✅ All nodes defined!")

## Part 3: Building the Graph

Now we connect everything using **Edges**.

In [ ]:
# Create the graph
workflow = StateGraph(OnboardingState)

# Add nodes
workflow.add_node("validate", validate_employee)
workflow.add_node("assign_equipment", assign_equipment)
workflow.add_node("setup_accounts", setup_accounts)
workflow.add_node("send_welcome", send_welcome_email)

# Add edges (define the flow)
workflow.add_edge(START, "validate")
workflow.add_edge("validate", "assign_equipment")
workflow.add_edge("assign_equipment", "setup_accounts")
workflow.add_edge("setup_accounts", "send_welcome")
workflow.add_edge("send_welcome", END)

# Compile the graph
onboarding_app = workflow.compile()

print("✅ Onboarding workflow created!")
print("\nWorkflow: START → validate → assign_equipment → setup_accounts → send_welcome → END")

## Part 4: Visualize the Graph (Optional)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(onboarding_app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization not available: {e}")
    print("But the workflow works perfectly!")

## Part 5: Run the Workflow

In [ ]:
# Test the workflow
initial_state = {
    "employee_name": "Karan Singh",
    "employee_id": "106",
    "department": "Engineering",
    "messages": [],
    "validation_status": "",
    "tasks_completed": []
}

print("="*70)
print("STARTING ONBOARDING WORKFLOW")
print("="*70)

result = onboarding_app.invoke(initial_state)

print("\n" + "="*70)
print("ONBOARDING COMPLETE!")
print("="*70)
print(f"\nEmployee: {result['employee_name']}")
print(f"Status: {result['validation_status']}")
print(f"Tasks completed: {len(result['tasks_completed'])}")
print(f"\nAll steps:")
for task in result['tasks_completed']:
    print(f"  ✓ {task}")

## 🎯 Exercise 1.1: Add an Orientation Node

Add a new node `schedule_orientation` that:
- Comes after `send_welcome` and before `END`
- Adds "Orientation scheduled" to tasks_completed
- Prints orientation date (e.g., "Orientation scheduled for [next Monday]")

In [ ]:
# Your code here
def schedule_orientation(state: OnboardingState):
    """TODO: Implement this node"""
    pass

# TODO: Create new workflow with the orientation node
# TODO: Test it

---
# Lab 2: Conditional Routing - Smart Leave Approval

**Objective:** Learn conditional edges to route based on state.

**Scenario:** Automatically route leave requests based on duration:
- Short leaves (≤3 days) → Auto-approved
- Long leaves (>3 days) → Manager review
- Insufficient balance → Rejected

**Key Concept:** Conditional edges allow dynamic routing based on state

## Setup - Employee Database

In [ ]:
from typing import Literal

# Employee database from Module 1
employees = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "manager": "102"},
    "102": {"name": "Rahul Verma", "department": "Engineering", "manager": "103"},
    "103": {"name": "Anjali Patel", "department": "HR", "manager": None},
    "104": {"name": "Arjun Reddy", "department": "Sales", "manager": "103"},
    "105": {"name": "Sneha Gupta", "department": "Marketing", "manager": "103"}
}

leave_balances = {
    "101": 12, "102": 8, "103": 15, "104": 10, "105": 5
}

print("✅ Employee database loaded!")

## Part 1: Define State

In [ ]:
class LeaveRequestState(TypedDict):
    """State for leave approval workflow"""
    employee_id: str
    employee_name: str
    days_requested: int
    leave_balance: int
    reason: str
    approval_status: str
    messages: Annotated[list, add_messages]
    requires_manager_approval: bool

print("✅ LeaveRequestState defined!")

## Part 2: Create Nodes

In [ ]:
def check_leave_balance(state: LeaveRequestState):
    """Node: Check if employee has enough leave"""
    print(f"\n📊 Checking leave balance for {state['employee_name']}...")
    
    employee_id = state['employee_id']
    balance = leave_balances.get(employee_id, 0)
    days = state['days_requested']
    
    if days > balance:
        return {
            "leave_balance": balance,
            "approval_status": "❌ Rejected - Insufficient balance",
            "messages": [("assistant", f"Request denied: You have {balance} days, but requested {days} days")]
        }
    
    return {
        "leave_balance": balance,
        "messages": [("assistant", f"Balance check passed: {balance} days available, {days} days requested")]
    }

def auto_approve(state: LeaveRequestState):
    """Node: Auto-approve short leaves (≤3 days)"""
    print(f"✅ Auto-approving leave for {state['employee_name']}...")
    
    return {
        "approval_status": "✅ Auto-Approved",
        "messages": [("assistant", f"Leave request for {state['days_requested']} days has been auto-approved!")]
    }

def manager_review(state: LeaveRequestState):
    """Node: Send to manager for approval (>3 days)"""
    print(f"👔 Sending to manager for review...")
    
    emp_id = state['employee_id']
    manager_id = employees.get(emp_id, {}).get("manager", "Unknown")
    manager_name = employees.get(manager_id, {}).get("name", "Manager")
    
    return {
        "approval_status": "⏳ Pending Manager Approval",
        "messages": [("assistant", f"Leave request for {state['days_requested']} days sent to {manager_name} for approval")]
    }

def reject_request(state: LeaveRequestState):
    """Node: Reject request due to insufficient balance"""
    print(f"❌ Rejecting leave request...")
    return state  # Status already set in check_leave_balance

print("✅ All nodes defined!")

## Part 3: Create Routing Function

**This is the key concept!** A routing function decides which node to go to next based on the state.

In [ ]:
def route_after_balance_check(state: LeaveRequestState) -> Literal["approve", "manager", "reject"]:
    """
    Routing function: Decides next node based on state
    
    Returns:
        - "reject" if insufficient balance
        - "approve" if ≤3 days and sufficient balance
        - "manager" if >3 days and sufficient balance
    """
    # Check if already rejected due to insufficient balance
    if "Rejected" in state.get("approval_status", ""):
        return "reject"
    
    # Route based on duration
    days = state['days_requested']
    if days <= 3:
        return "approve"
    else:
        return "manager"

print("✅ Routing function defined!")
print("\nRouting logic:")
print("  - Insufficient balance → reject")
print("  - ≤3 days + sufficient → approve")
print("  - >3 days + sufficient → manager")

## Part 4: Build Graph with Conditional Routing

In [ ]:
# Create workflow
leave_workflow = StateGraph(LeaveRequestState)

# Add nodes
leave_workflow.add_node("check_balance", check_leave_balance)
leave_workflow.add_node("approve", auto_approve)
leave_workflow.add_node("manager", manager_review)
leave_workflow.add_node("reject", reject_request)

# Set entry point
leave_workflow.add_edge(START, "check_balance")

# Add conditional edge - THIS IS THE KEY!
leave_workflow.add_conditional_edges(
    "check_balance",  # From this node
    route_after_balance_check,  # Use this function to decide
    {  # Map return values to node names
        "approve": "approve",
        "manager": "manager",
        "reject": "reject"
    }
)

# All paths lead to END
leave_workflow.add_edge("approve", END)
leave_workflow.add_edge("manager", END)
leave_workflow.add_edge("reject", END)

# Compile
leave_app = leave_workflow.compile()

print("✅ Leave approval workflow with conditional routing created!")

## Part 5: Test Different Scenarios

In [ ]:
# Test cases
test_cases = [
    # Case 1: Short leave with sufficient balance → Auto-approve
    {
        "employee_id": "101",
        "employee_name": "Priya Sharma",
        "days_requested": 2,
        "reason": "Family function",
        "messages": [],
        "approval_status": "",
        "leave_balance": 0,
        "requires_manager_approval": False
    },
    # Case 2: Long leave with sufficient balance → Manager review
    {
        "employee_id": "102",
        "employee_name": "Rahul Verma",
        "days_requested": 5,
        "reason": "Vacation",
        "messages": [],
        "approval_status": "",
        "leave_balance": 0,
        "requires_manager_approval": False
    },
    # Case 3: Insufficient balance → Reject
    {
        "employee_id": "105",
        "employee_name": "Sneha Gupta",
        "days_requested": 7,
        "reason": "Personal",
        "messages": [],
        "approval_status": "",
        "leave_balance": 0,
        "requires_manager_approval": False
    }
]

for i, test_case in enumerate(test_cases, 1):
    print("\n" + "="*70)
    print(f"TEST CASE {i}: {test_case['employee_name']} - {test_case['days_requested']} days")
    print("="*70)
    
    result = leave_app.invoke(test_case)
    
    print(f"\n📋 Final Status: {result['approval_status']}")
    print(f"💼 Employee: {result['employee_name']}")
    print(f"📅 Days Requested: {result['days_requested']}")
    print(f"🏦 Leave Balance: {result['leave_balance']}")

## 🎯 Exercise 2.1: Add Emergency Leave Path

Modify the routing function to add a new path:
- If reason contains "emergency" → Auto-approve regardless of days
- Add an `emergency_approve` node
- Update the routing function and graph

In [ ]:
# Your code here
def emergency_approve(state: LeaveRequestState):
    """TODO: Implement emergency approval"""
    pass

# TODO: Update routing function
# TODO: Rebuild graph with emergency path

---
# Lab 3: Parallel Execution

**Objective:** Learn how to execute multiple nodes in parallel.

**Scenario:** During employee onboarding, some tasks can happen simultaneously:
- IT setup (equipment + accounts)
- HR paperwork
- Facilities access

**Key Concept:** Parallel execution speeds up workflows when tasks are independent

## Part 1: Define State for Parallel Workflow

In [ ]:
import time

class ParallelOnboardingState(TypedDict):
    """State for parallel onboarding workflow"""
    employee_name: str
    employee_id: str
    department: str
    it_setup_complete: bool
    hr_paperwork_complete: bool
    facilities_complete: bool
    messages: Annotated[list, add_messages]

print("✅ ParallelOnboardingState defined!")

## Part 2: Create Parallel Nodes

These nodes will execute simultaneously!

In [ ]:
def it_setup(state: ParallelOnboardingState):
    """Node: IT Setup (takes 2 seconds)"""
    print(f"💻 Starting IT setup for {state['employee_name']}...")
    time.sleep(2)  # Simulate work
    print(f"✅ IT setup complete!")
    
    return {
        "it_setup_complete": True,
        "messages": [("assistant", "IT setup: Laptop, email, and accounts configured")]
    }

def hr_paperwork(state: ParallelOnboardingState):
    """Node: HR Paperwork (takes 2 seconds)"""
    print(f"📄 Starting HR paperwork for {state['employee_name']}...")
    time.sleep(2)  # Simulate work
    print(f"✅ HR paperwork complete!")
    
    return {
        "hr_paperwork_complete": True,
        "messages": [("assistant", "HR paperwork: Contracts, policies, and benefits enrolled")]
    }

def facilities_access(state: ParallelOnboardingState):
    """Node: Facilities Access (takes 2 seconds)"""
    print(f"🏢 Setting up facilities access for {state['employee_name']}...")
    time.sleep(2)  # Simulate work
    print(f"✅ Facilities access complete!")
    
    return {
        "facilities_complete": True,
        "messages": [("assistant", "Facilities: Building access card and parking assigned")]
    }

def finalize_onboarding(state: ParallelOnboardingState):
    """Node: Finalize after all parallel tasks complete"""
    print(f"\n🎉 Finalizing onboarding for {state['employee_name']}...")
    
    return {
        "messages": [("assistant", f"All onboarding tasks complete! Welcome aboard, {state['employee_name']}!")]
    }

print("✅ All parallel nodes defined!")

## Part 3: Build Graph with Parallel Execution

In [ ]:
# Create workflow
parallel_workflow = StateGraph(ParallelOnboardingState)

# Add nodes
parallel_workflow.add_node("it_setup", it_setup)
parallel_workflow.add_node("hr_paperwork", hr_paperwork)
parallel_workflow.add_node("facilities_access", facilities_access)
parallel_workflow.add_node("finalize", finalize_onboarding)

# Connect START to all three parallel nodes
parallel_workflow.add_edge(START, "it_setup")
parallel_workflow.add_edge(START, "hr_paperwork")
parallel_workflow.add_edge(START, "facilities_access")

# All parallel nodes connect to finalize
parallel_workflow.add_edge("it_setup", "finalize")
parallel_workflow.add_edge("hr_paperwork", "finalize")
parallel_workflow.add_edge("facilities_access", "finalize")

# Finalize connects to END
parallel_workflow.add_edge("finalize", END)

# Compile
parallel_app = parallel_workflow.compile()

print("✅ Parallel onboarding workflow created!")
print("\n📊 Workflow structure:")
print("         START")
print("        /  |  \\")
print("      IT  HR  Facilities")
print("        \\  |  /")
print("       Finalize")
print("          |")
print("         END")

## Part 4: Test Parallel Execution

Watch how all three tasks run simultaneously!

In [ ]:
import time

initial_state = {
    "employee_name": "Vikram Mehta",
    "employee_id": "110",
    "department": "Sales",
    "it_setup_complete": False,
    "hr_paperwork_complete": False,
    "facilities_complete": False,
    "messages": []
}

print("="*70)
print("STARTING PARALLEL ONBOARDING")
print("="*70)
print("⏱️  Each task takes 2 seconds")
print("🚀 Running in parallel...\n")

start_time = time.time()
result = parallel_app.invoke(initial_state)
end_time = time.time()

print("\n" + "="*70)
print("PARALLEL ONBOARDING COMPLETE!")
print("="*70)
print(f"\n⏱️  Total time: {end_time - start_time:.2f} seconds")
print("📊 Notice: ~2 seconds total, not 6 seconds!")
print("\n✅ Status:")
print(f"  - IT Setup: {result['it_setup_complete']}")
print(f"  - HR Paperwork: {result['hr_paperwork_complete']}")
print(f"  - Facilities: {result['facilities_complete']}")

## 🎯 Exercise 3.1: Add Background Check

Add a new parallel task:
1. Create `background_check(state)` node (simulate 3 seconds)
2. Add it to the parallel execution
3. Update the state to track background_check_complete
4. Compare total execution time

In [ ]:
# Your code here
def background_check(state: ParallelOnboardingState):
    """TODO: Implement background check"""
    pass

# TODO: Update state schema
# TODO: Rebuild graph with background check

---
# Lab 4: Combining LangGraph with LangChain Agents

**Objective:** Use LangChain agents within LangGraph nodes.

**Scenario:** Create an HR assistant that uses tools within a workflow.

**Key Concept:** Agents can be nodes in a graph!

## Part 1: Create Tools (LangChain 1.0 Way)

In [ ]:
from langchain_core.tools import tool

@tool
def check_leave_balance_tool(employee_id: str) -> str:
    """Check the leave balance for an employee.
    
    Args:
        employee_id: The employee's ID number
    
    Returns:
        A message with the employee's leave balance
    """
    balance = leave_balances.get(employee_id, 0)
    return f"Employee {employee_id} has {balance} days of leave available."

@tool
def get_employee_info_tool(employee_id: str) -> str:
    """Get information about an employee.
    
    Args:
        employee_id: The employee's ID number
    
    Returns:
        Employee information including name and department
    """
    emp = employees.get(employee_id, {})
    if emp:
        return f"Name: {emp['name']}, Department: {emp['department']}"
    return f"Employee {employee_id} not found."

print("✅ Tools created!")

## Part 2: Create Agent (LangChain 1.0 - Updated!)

**Note:** Using `system_prompt` parameter instead of `prompt` (LangChain 1.0 change)

In [ ]:
from langchain.agents import create_agent

# Create agent with system_prompt (NEW in LangChain 1.0)
hr_agent = create_agent(
    model=llm,
    tools=[check_leave_balance_tool, get_employee_info_tool],
    system_prompt="""You are a helpful HR assistant. 
    Use the available tools to help employees with:
    - Checking leave balances
    - Getting employee information
    
    Be friendly and professional in your responses."""
)

print("✅ HR Agent created with system_prompt (LangChain 1.0)!")

## Part 3: Use Agent in a LangGraph Node

In [ ]:
class HRWorkflowState(TypedDict):
    """State for HR workflow with agent"""
    user_query: str
    agent_response: str
    messages: Annotated[list, add_messages]

def agent_node(state: HRWorkflowState):
    """Node that runs the agent"""
    print(f"\n🤖 Processing query: {state['user_query']}")
    
    # Invoke agent with user query
    response = hr_agent.invoke({
        "messages": [{"role": "user", "content": state['user_query']}]
    })
    
    # Extract the agent's response
    agent_response = response["messages"][-1].content
    
    return {
        "agent_response": agent_response,
        "messages": [("assistant", agent_response)]
    }

# Create simple workflow
agent_workflow = StateGraph(HRWorkflowState)
agent_workflow.add_node("agent", agent_node)
agent_workflow.add_edge(START, "agent")
agent_workflow.add_edge("agent", END)

agent_app = agent_workflow.compile()

print("✅ Agent integrated into LangGraph!")

## Part 4: Test the Agent Workflow

In [ ]:
# Test queries
queries = [
    "What's the leave balance for employee 101?",
    "Tell me about employee 103",
    "Check leave balance and info for employee 102"
]

for query in queries:
    print("\n" + "="*70)
    result = agent_app.invoke({
        "user_query": query,
        "agent_response": "",
        "messages": []
    })
    print(f"\n📋 Response:\n{result['agent_response']}")

---
## Summary

### What You Learned:

1. **Lab 1 - Linear Workflows:**
   - State, Nodes, Edges, START, END
   - Building sequential processes

2. **Lab 2 - Conditional Routing:**
   - Routing functions that return node names
   - `add_conditional_edges()` for dynamic flows
   - Decision logic based on state

3. **Lab 3 - Parallel Execution:**
   - Multiple edges from START
   - Independent tasks running simultaneously
   - Performance benefits of parallelization

4. **Lab 4 - Agents in Graphs:**
   - **LangChain 1.0 Change:** Use `system_prompt` instead of `prompt`
   - Agents as nodes in workflows
   - Combining structured workflows with intelligent agents

### Key LangChain 1.0 Update:

```python
# OLD (pre-1.0) - ❌ Will error in 1.0
agent = create_agent(model, tools, prompt=prompt_template)

# NEW (1.0) - ✅ Correct syntax
agent = create_agent(
    model=llm,
    tools=[...],
    system_prompt="You are a helpful assistant..."
)
```

### Next Steps:
- Complete all exercises
- Combine concepts (e.g., parallel + conditional)
- Build your own HR workflow
- Explore human-in-the-loop patterns